In [11]:
import numpy as np
import antea.database.load_db as db
from antea.io.mc_io import load_mcsns_response
from antea.io.mc_io import load_mcTOFsns_response
from antea.io.mc_io import load_mcparticles
from antea.io.mc_io import load_mchits

import antea.reco.reco_functions   as rf

In [12]:
filename     = '/Users/carmenromoluque/nexus_petit_analysis/PETit-ring/ring_dataframes_analysis/petit_ring_tof_all_tables.000.pet.h5'

In [14]:
DataSiPM     = db.DataSiPM('petalo', 0)
DataSiPM_idx = DataSiPM.set_index('SensorID')
sns_response = load_mcsns_response(filename)
tof_response = load_mcTOFsns_response(filename)
particles    = load_mcparticles(filename)
hits         = load_mchits(filename)
events       = particles.event_id.unique()

sipms        = DataSiPM_idx.loc[sns_response.sensor_id]
sns_ids      = sipms.index.values

charge_range = (1000, 1400)

for evt in events[:100]:
    evt_sns = sns_response[sns_response.event_id == evt]
    evt_sns = rf.find_SiPMs_over_threshold(evt_sns, threshold=2)
    if len(evt_sns) == 0:
        continue

    evt_parts = particles[particles.event_id       == evt]
    evt_hits  = hits[hits.event_id                 == evt]
    evt_tof   = tof_response[tof_response.event_id == evt]

    pos1, pos2, q1, q2, _, _, _, _, min_id1, min_id2, min_tof1, min_tof2 = rf.reconstruct_coincidences(evt_sns, evt_tof, charge_range, DataSiPM_idx, evt_parts, evt_hits)
    
    if len(pos1) and len(pos2):
        print(evt)

31
